In [1]:
import pandas as pd

In [2]:
path_to_old = r"E:\backup\backup_BPT\GAMA_ETG_OLA.csv"
path_to_dr3 = r"E:\backup\backup_BPT\GAMA_ETG_OLA3.csv"

old_df = pd.read_csv(path_to_old)
dr3_df = pd.read_csv(path_to_dr3)

1. Downgrade the spectra catalogue from DR4 to DR3 does not change the sample and classification of Ola's sample - it is still 1995 galaxies

In [3]:
DataFrame = pd.merge(old_df, dr3_df, how='inner', on='SPECID')

print(DataFrame.info)

df = DataFrame.query("BPT_x != BPT_y")
print(df.shape)

df = DataFrame.query("WHAN_x != WHAN_y")
print(df.shape)

<bound method DataFrame.info of                    SPECID  CATAID_1_x SURVEY_x  IS_BEST_x  IS_SBEST_x  \
0     1031417538174543872      463196     SDSS       True        True   
1     1031419462319892480      491454     SDSS       True        True   
2     1031421386465241088      484660     SDSS       True        True   
3     1031429357924542464      460399     SDSS       True        True   
4     1031457395471050752      460542     SDSS       True        True   
...                   ...         ...      ...        ...         ...   
1990       G15_Y6_101_299       64569     GAMA       True        True   
1991       G15_Y6_101_321       64674     GAMA       True        True   
1992       G15_Y6_101_329       64666     GAMA       True        True   
1993       G15_Y6_102_233      266085     GAMA       True        True   
1994       G15_Y6_104_144       92876     GAMA       True        True   

           RA_x    DEC_x       Z_x  BPT_x WHAN_x  ...    HA_er_y         HB_y  \
0     211.

2. Merge all required DR4 catalogues into one and create new sample of dusty ETGs 

In [4]:
DirectSummation_path = r"E:\databases\GAMAs4\DirectSummationv05"
DS_cols ={
    'SPECID' : 0,
    'CATAID' : 1,
    'RA' : 2,
    'DEC' : 3,
    'Z' : 4,
    'SURVEY' : 6,
    'IS_BEST' : 8,
    'IS_SBEST' : 9,
    'NIIR_FLUX_ERR': 71,
    'NIIR_FLUX' : 72,
    'HA_EW_ERR' : 79,
    'HA_EW' : 80,
    'HA_FLUX_ERR' : 81,
    'HA_FLUX' : 82,
    'OIIIR_FLUX_ERR' : 166,
    'OIIIR_FLUX' : 167,
    'HB_FLUX_ERR' : 181,
    'HB_FLUX' : 182
    }
DirectSummation = pd.read_csv(DirectSummation_path, sep=r"\s+", engine='python', usecols=DS_cols.values(), names=DS_cols.keys())
DirectSummation = DirectSummation.query('IS_BEST == True & IS_SBEST == True & (SURVEY == "SDSS" | SURVEY == "GAMA")')
# DirectSummation = DirectSummation.query('IS_BEST == True & IS_SBEST == True')

MagPhys_path = r'E:\databases\GAMAs4\MagPhysv06'
MP_cols = {
    'CATAID' : 0,
    'mass_stellar_percentile16' : 34,
    'mass_stellar_percentile50' : 35,
    'mass_stellar_percentile84' : 36,
    'T_C_ISM_percentile16' : 46,
    'T_C_ISM_percentile50' : 47,
    'T_C_ISM_percentile84' : 48,
    'T_W_BC_percentile16' : 52, 
    'T_W_BC_percentile50' : 53, 
    'T_W_BC_percentile84' : 54, 
    'mass_dust_percentile16' : 88,
    'mass_dust_percentile50' : 89,
    'mass_dust_percentile84' : 90,
    'SFR_0_1Gyr_percentile16' : 94,
    'SFR_0_1Gyr_percentile50' : 95,
    'SFR_0_1Gyr_percentile84' : 96,
    'ager_percentile16' : 124,
    'ager_percentile50' : 125,
    'ager_percentile84' : 126,
    }
MagPhys = pd.read_csv(MagPhys_path, sep=r"\s+", engine='python', usecols=MP_cols.values(), names=MP_cols.keys())
MagPhys.info
#add age, temperature and mass of dust to MagPhys

Sersic_path = r'E:\databases\GAMAs4\SersicCatSDSSv09'
S_cols = {
    'CATAID' : 0,
    'GALRE_r' : 85,
    'GALINDEX_r' : 86,
    'GALREERR_r' : 92,
    'GALINDEXERR_r' : 93,
    }
Sersic = pd.read_csv(Sersic_path, sep=r"\s+", engine='python', usecols=S_cols.values(), names=S_cols.keys())

LambdaR_path = r'E:\databases\GAMAs4\LambdarCatv01'
LambdaR_cols = {
    'CATAID' : 0,
    'W1_flux' : 27,                   
    'W1_fluxerr' : 28,  
    'W2_flux' : 29,   
    'W2_fluxerr' : 30,  
    'W3_flux' : 31,                   
    'W3_fluxerr' : 32,
    'W4_flux' : 33, 
    'W4_fluxerr' : 34,
    'P100_flux' : 35,
    'P100_fluxerr' : 36,
    'S250_flux' : 39,
    'S250_fluxerr' : 40
}
LambdaR = pd.read_csv(LambdaR_path, sep=r"\s+", engine='python', usecols=LambdaR_cols.values(), names=LambdaR_cols.keys())

Env_path = r"E:\databases\GAMAs4\EnvironmentMeasuresv06.txt"
Env_cols = {
    'CATAID' : 0,
    'SurfaceDensity' : 9,
    'SurfaceDensityErr' : 10,
    'SurfaceDensityFlag' : 11
}
Env = pd.read_csv(Env_path, sep=r"\s+", engine='python', usecols=Env_cols.values(), names=Env_cols.keys())

In [5]:
from functools import reduce

frames = [DirectSummation, MagPhys, Sersic, LambdaR, Env]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['CATAID'],
                                            how='left'), frames)

df_merged_path = r'E:\databases\GAMAs4\GAMA_DR4.csv'
df_merged.info()
df_merged.fillna(-99999.0, inplace=True)
df_merged.to_csv(df_merged_path, index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 287739 entries, 0 to 287738
Data columns (total 55 columns):
 #   Column                     Non-Null Count   Dtype  
---  ------                     --------------   -----  
 0   SPECID                     287739 non-null  object 
 1   CATAID                     287739 non-null  int64  
 2   RA                         287739 non-null  float64
 3   DEC                        287739 non-null  float64
 4   Z                          287739 non-null  float64
 5   SURVEY                     287739 non-null  object 
 6   IS_BEST                    287739 non-null  bool   
 7   IS_SBEST                   287739 non-null  bool   
 8   NIIR_FLUX_ERR              287739 non-null  float64
 9   NIIR_FLUX                  287739 non-null  float64
 10  HA_EW_ERR                  287739 non-null  float64
 11  HA_EW                      287739 non-null  float64
 12  HA_FLUX_ERR                287739 non-null  float64
 13  HA_FLUX                    28

In [6]:
dataframe_ETGs = df_merged.query("(S250_flux > 3*S250_fluxerr & S250_flux > 0 & S250_fluxerr > 0) & GALINDEX_r > 4 & Z > 0.01 & Z < 0.32 & mass_stellar_percentile50 > 0")
# dataframe_ETGs = df_merged.query("(S250_flux > 3*S250_fluxerr & S250_flux > 0 & S250_fluxerr > 0) & GALINDEX_r > 4 & Z > 0.01 & Z < 0.32")
dataframe_ETGs_path = r'E:\databases\GAMAs4\DETG_DR4.csv'
dataframe_ETGs.to_csv(dataframe_ETGs_path, index=False)

In [7]:
from full_GAMA import *

dataframe_ETGs_path = r'E:\databases\GAMAs4\DETG_DR4.csv'
dataframe_ETGs = pd.read_csv(dataframe_ETGs_path)
dataframe_ETGs.reset_index(inplace=True, drop=True)
DataFrame = dataframe_ETGs

MS_flag = MS_flagging(DataFrame['SFR_0_1Gyr_percentile50'], DataFrame['mass_stellar_percentile50'], DataFrame['Z'])
bMS_MS = [1 if item > -0.2 else 0 for item in MS_flag]
apers = aperture_flagging(DataFrame['GALRE_r'], DataFrame['GALREERR_r'], DataFrame['SURVEY'])

BPTs = []
WHANs = []
LAGNs = []
LAGN_ers = []
OUTFLOW = []
OUTFLOW_up = []
OUTFLOW_down = []
OUTFLOW_off = []
OUTFLOW_up_off = []
OUTFLOW_down_off = []
        
for i in range(len(DataFrame['OIIIR_FLUX'])):
    BPT, WHANx, LOIII, LOIII_er = spectra_processing(DataFrame['OIIIR_FLUX'][i], DataFrame['OIIIR_FLUX_ERR'][i], DataFrame['HB_FLUX'][i], DataFrame['HB_FLUX_ERR'][i],
                                                          DataFrame['NIIR_FLUX'][i], DataFrame['NIIR_FLUX_ERR'][i], DataFrame['HA_FLUX'][i], DataFrame['HA_FLUX_ERR'][i],
                                                          DataFrame['HA_EW'][i], DataFrame['HA_EW_ERR'][i], DataFrame['Z'][i])
            
    if LOIII == -99999.0 or DataFrame['SFR_0_1Gyr_percentile50'][i] == -99999.0 or DataFrame['mass_stellar_percentile50'][i] == -99999.0:
        OUTFLOW_up.append(-99999.0)
        OUTFLOW.append(-99999.0)
        OUTFLOW_down.append(-99999.0)
    else:    
        if LOIII_er < 0:
            OUTFLOW_up.append(-100000.0)
            OUTFLOW.append(outflow_wAGN(DataFrame['SFR_0_1Gyr_percentile50'][i], LOIII, DataFrame['mass_stellar_percentile50'][i]))
            OUTFLOW_down.append(-100000.0)
        else:
            OUTFLOW_up.append(outflow_wAGN(DataFrame['SFR_0_1Gyr_percentile50'][i], np.log10(10**LOIII + 10**LOIII_er), DataFrame['mass_stellar_percentile50'][i]))
            OUTFLOW.append(outflow_wAGN(DataFrame['SFR_0_1Gyr_percentile50'][i], LOIII, DataFrame['mass_stellar_percentile50'][i]))
            OUTFLOW_down.append(outflow_wAGN(DataFrame['SFR_0_1Gyr_percentile50'][i], np.log10(10**LOIII - 10**LOIII_er), DataFrame['mass_stellar_percentile50'][i]))
            
    if DataFrame['SFR_0_1Gyr_percentile50'][i] == -99999.0 or DataFrame['mass_stellar_percentile50'][i] == -99999.0:
        OUTFLOW_up_off.append(-99999.0)
        OUTFLOW_off.append(-99999.0)
        OUTFLOW_down_off.append(-99999.0) 
    else:               
        OUTFLOW_up_off.append(outflow_woAGN(DataFrame['SFR_0_1Gyr_percentile84'][i], DataFrame['mass_stellar_percentile16'][i]))
        OUTFLOW_off.append(outflow_woAGN(DataFrame['SFR_0_1Gyr_percentile50'][i], DataFrame['mass_stellar_percentile50'][i]))
        OUTFLOW_down_off.append(outflow_woAGN(DataFrame['SFR_0_1Gyr_percentile16'][i], DataFrame['mass_stellar_percentile84'][i]))
            
    BPTs.append(BPT)
    WHANs.append(WHANx)
    LAGNs.append(LOIII)
    LAGN_ers.append(LOIII_er)


DataFrame.insert(8, "BPT", BPTs, True)
DataFrame.insert(8, "WHAN", WHANs, True)
DataFrame["LOIII"] = LAGNs
DataFrame["LOIII_er"] = LAGN_ers
DataFrame["out_on_16"] = OUTFLOW_down
DataFrame["out_on_50"] = OUTFLOW
DataFrame["out_on_84"] = OUTFLOW_up
DataFrame["out_off_16"] = OUTFLOW_down_off
DataFrame["out_off_50"] = OUTFLOW_off
DataFrame["out_off_84"] = OUTFLOW_up_off
DataFrame["deltaMS"] = MS_flag
DataFrame["BMS"] = bMS_MS
DataFrame["in_aperture"] = apers
DataFrame["mdms_percentile16"] = DataFrame["mass_dust_percentile16"] - DataFrame["mass_stellar_percentile84"]
DataFrame["mdms_percentile50"] = DataFrame["mass_dust_percentile50"] - DataFrame["mass_stellar_percentile50"]
DataFrame["mdms_percentile84"] = DataFrame["mass_dust_percentile84"] - DataFrame["mass_stellar_percentile16"]

DataFrame = DataFrame.query('BPT != "NDA" & WHAN != "NDA"')
dataframe_ETGs_path = r'E:\databases\GAMAs4\DETG_DR4.csv'
DataFrame.to_csv(dataframe_ETGs_path, index=False)

In [15]:
dataframe_ETGs_path = r'E:\databases\GAMAs4\DETG_DR4.csv'
dataframe_ETGs = pd.read_csv(dataframe_ETGs_path)

# problematic_data = dataframe_ETGs.query('(SURVEY != "GAMA" & SURVEY != "SDSS") | mass_stellar_percentile50 < 0 | (WHAN == "NDA" | BPT == "NDA")')
problematic_data = dataframe_ETGs.query('in_aperture == 0')
print(np.percentile(problematic_data['mass_stellar_percentile50'], 50), np.percentile(problematic_data['mass_stellar_percentile50'], 50) - np.percentile(problematic_data['mass_stellar_percentile50'], 16), np.percentile(problematic_data['mass_stellar_percentile50'], 84) - np.percentile(problematic_data['mass_stellar_percentile50'], 50))
print(problematic_data.shape)

# problematic_data_path = r'E:\databases\GAMAs4\PROBS.csv'
# problematic_data.to_csv(problematic_data_path, index=False)

10.7 0.4299999999999997 0.3100000000000005
(1202, 71)
